In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2

import warnings
warnings.filterwarnings('ignore')

from PIL import Image
import tensorflow as tf

from sklearn.model_selection import train_test_split
from skimage.transform import resize
from sklearn.metrics import accuracy_score

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

# 1. Loading the dataset

In [ ]:
# loading dataset
data = []
labels = []
classes = 43
cur_path = '../input/gtsrb-german-traffic-sign/Train'

for i in os.listdir(cur_path):
    dir = cur_path + '/' + i
    for j in os.listdir(dir):
        img_path = dir+'/'+j
        img = cv2.imread(img_path,-1)
        img = cv2.resize(img, (30,30), interpolation = cv2.INTER_NEAREST)
        data.append(img)
        labels.append(i)
        
data = np.array(data)
labels = np.array(labels)
print(data.shape, labels.shape)

In [ ]:
i_path = '../input/gtsrb-german-traffic-sign/Train/1/00001_00072_00027.png'
plt.imshow(cv2.imread(i_path, -1))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size= 0.25, random_state=21)

print((X_train.shape, y_train.shape), (X_test.shape, y_test.shape))

# converting the labels into one hot encoding
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

# 2. Visualizing the data

In [ ]:
# number of images in each class
data_dic = {}
for folder in os.listdir(cur_path):
    data_dic[folder] = len(os.listdir(cur_path + '/' + folder))

data_df= pd.Series(data_dic)
plt.figure(figsize = (15, 6))
data_df.sort_values().plot(kind = 'bar')
plt.xlabel('Classes')
plt.ylabel('Number of images')

# 3. Model

In [ ]:
# defining model structure
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5), activation ='relu', input_shape = (30,30,3)))
model.add(Conv2D(filters = 32, kernel_size = (5,5), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(rate = 0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3), activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3), activation ='relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(rate = 0.25))

model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(rate = 0.5))
model.add(Dense(43, activation= 'softmax'))

# model compilation
model.compile(loss = 'categorical_crossentropy', optimizer ='adam', metrics =['accuracy'])

In [ ]:
model.summary()

In [ ]:
!pip install visualkeras
import visualkeras

In [ ]:
visualkeras.layered_view(model)

In [ ]:
epochs = 15
history = model.fit(X_train, y_train, 
                   batch_size=64,
                   epochs=epochs,
                   validation_data=(X_test, y_test))

# 4. Plots

In [ ]:
plt.figure(0)
plt.plot(history.history['accuracy'], label= 'train accuracy')
plt.plot(history.history['val_accuracy'], label= 'test accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('Accuracy')
plt.legend()

In [ ]:
plt.figure(0)
plt.plot(history.history['loss'], label= 'train loss')
plt.plot(history.history['val_loss'], label= 'test loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('Loss')
plt.legend()

# 5. Model testing on test dataset

In [ ]:
y_test = pd.read_csv('../input/gtsrb-german-traffic-sign/Test.csv')

In [ ]:
plt.imshow(cv2.imread('../input/gtsrb-german-traffic-sign/Test/00000.png'))

In [ ]:
labels_test = y_test['ClassId'].values
img_test = y_test['Path'].values
test_dir = '../input/gtsrb-german-traffic-sign'
data = []
for img in img_test:
    img_path = os.path.join(test_dir, img)
#     print(img_path)
    image = cv2.imread(img_path, -1)
    image = cv2.resize(image, (30,30), interpolation = cv2.INTER_NEAREST)
    data.append(np.array(image))
    
X_test = np.array(data)

In [ ]:
pred = model.predict_classes(X_test)

print("Test accuracy: ", accuracy_score(labels_test, pred) * 100 )

# Model has 96% accuracy on the test data